In [3]:
import pandas as pd
import os 
os.getcwd()

'/Users/carolinesofieljorring/Desktop/Social data analysis and visualization/socialdata2022-main/project'

In [28]:
# enhedsomraade 
cols = ['kategori','id','ny_dm_element','traeart','slaegtsnavn','planteaar','faeldningsaar','bydelsnavn','byrumstype','vejnavn','fritrumsprofil','stammeomfang','placering','omgivelse','reg_dato','opdateret_dato','wkb_geometry']
df = pd.read_csv('trae_basis.csv',sep=',',usecols=cols)
#drop_columns = ['FID','under_element','arbsted_tekst_id','plantet_halvaar','vaekstsaeson','underordnet_stednavn','omraade','kronediameter','vaekstmedie','beskyttelse','sundhed','skudtilsvaekst','andre_vaekstforhold','bevaringsvaerdi','driftstatus_gadetrae','garantiperioden_udloeber','vanding_i_aar','ejer','driftsorganisation','driftsorganisation_vanding','driftsaftale_edoc_nr','']
df.head()

,kategori,id,ny_dm_element,traeart,slaegtsnavn,planteaar,faeldningsaar,bydelsnavn,byrumstype,vejnavn,fritrumsprofil,stammeomfang,placering,omgivelse,reg_dato,opdateret_dato,wkb_geometry
0,gadetræ,2755,Gade- og allétræer,Platanus hybr. acerifolia,Platan,2008.0,Ikke registreret,Vanløse,Gader og pladser,Sallingvej,"4,5 m",40 - 60,Siderabat,Fast belægning,2011-07-01T00:00:00,2022-03-21T10:32:29.888,POINT (12.506472982261277 55.69325544860847)
1,gadetræ,79754,Gade- og allétræer,Tilia hybr. europaea 'Pallida',Lind,2005.0,Ikke registreret,Valby,Gader og pladser,Mølle Allé,"4,5 m",18 - 20,Fortov,Fast belægning,2016-04-07T09:47:27.171,2022-03-21T10:32:29.888,POINT (12.517024058234247 55.665248690540835)
2,gadetræ,84790,Gade- og allétræer,Tilia hybr. europaea 'Pallida',Lind,2018.0,Ikke registreret,Valby,Gader og pladser,Mølle Allé,"4,5 m",18 - 20,Fortov,Fast belægning,2018-01-08T00:00:00,2022-03-21T10:32:29.888,POINT (12.517473257630915 55.6651562247192)
3,gadetræ,79761,Formede træer,Tilia hybr. europaea 'Pallida',Lind,1960.0,Ikke registreret,Valby,Gader og pladser,Mølle Allé,"4,5 m",18 - 20,Fortov,Fast belægning,2016-04-07T09:47:27.171,2022-03-21T10:32:29.888,POINT (12.515573415316043 55.665570503006286)
4,gadetræ,6881,Gade- og allétræer,Robinia pseudoacacia 'Nyirsegi',Robinie,2008.0,Ikke registreret,Vesterbro-Kongens Enghave,Gader og pladser,Dannebrogsgade,"4,5 m",18 - 20,Fortov,Fast belægning,2011-07-01T00:00:00,2022-03-09T17:16:03.502,POINT (12.552988210062669 55.669186278185045)


In [ ]:
# Idéen med datasættet, hvad vil vi undersøge?
# hvorfor vigtigt? Vigtigt med træer ift. klimaaflastning (CO2), god stemning i byer, træer/grønne områder skaber ro 

# Exploratory analysis
# Hvor mange træer i Kbh, hvor mange træer i de forskellige bydele
# Hvilke slags træer - overordnet, i de forskellige bydele 
# Hvornår er træerne plantet - i hvilke år plantes der flest 
# Hvor store er træerne? Forskel på størrelse ift. bydele 
# Geo plot, brug lat,loc